In [154]:
import numpy as np
import openml
# from scipy.io import arff

In [155]:
np.set_printoptions(precision=4, suppress=True)

In [156]:
data = openml.datasets.get_dataset(1590, download_data=True) # Pegando os dados via API do OpenML
X, Y, categorical , f = data.get_data(target=data.default_target_attribute, dataset_format='array')

C:\Users\Lorde\AppData\Local\Temp\ipykernel_36616\1127533313.py:2: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, Y, categorical , f = data.get_data(target=data.default_target_attribute, dataset_format='array')


In [157]:
X.shape, Y.shape

((48842, 14), (48842,))

In [158]:
X = X[:1001]
Y = Y[:1001]

In [159]:
features_is_categorical = list(zip(f, categorical))

In [160]:
rows_nan_indexes = np.where(np.isnan(X).any(axis=1))[0] ## Removendo os NANs

X = np.delete(X, rows_nan_indexes, axis=0)
Y = np.delete(Y, rows_nan_indexes, axis=0)
X.shape, Y.shape   

((927, 14), (927,))

# 1) Classificador KNN

In [161]:
class KNN: #Refazer diferente
    def __init__(self, k, method): 
        #construtor em python que recebe o tipo de saída do modelo e os k vizinhos
        self.k = k
        self.method = method
        
    def fit(self, X_train, y_train): #Atribui os dados as variáveis
        self.X_train = X_train
        self.y_train = y_train 
        
        
    def euclidean_distance(self, x1, x2): #Calcula distâncias euclidianas
        return np.sqrt(np.sum((x1 - x2) ** 2))
    
    def manhattan_distance(self, x1, x2): #Calcula distâncias de manhattan
        return np.sum( abs(x1 - x2) )
    
    def predict(self, X):

        predictions = []
        for x in X:
            # Calcula distâncias de x até todas as amostras de treino
            if self.method == "euclidean":
                distances = [self.euclidean_distance(x, x_train) for x_train in self.X_train]
            elif self.method == "manhattan":
                distances = [self.manhattan_distance(x, x_train) for x_train in self.X_train]

            # Pega índices dos k vizinhos mais próximos
            k_indices = np.argsort(distances)[:self.k]

            # Extrai as classes correspondentes
            k_nearest_labels = [self.y_train[i] for i in k_indices]

            # Conta a frequência de cada classe e pega a mais comum
            unique, counts = np.unique(k_nearest_labels, return_counts=True)
            predictions.append(unique[np.argmax(counts)])
            
            # print(predictions)

        return np.array(predictions)



# 2) Classificador Bayesiano

In [162]:
class NaiveBayesUnivariado:
    def __init__(self):
        pass
    
    def fit(self, X: np.ndarray, Y: np.ndarray):
        self.classes = np.unique(Y)
        n_samples, n_features = X.shape

        # Guardar valores em dicionários
        self.means = {}
        self.vars = {}
        self.priors = {}

        # Calcular média, Desvio Padrão e prior para cada classe
        for c in self.classes:
            X_c = X[Y == c]
            self.means[c] = X_c.mean(axis=0)
            self.vars[c] = X_c.var(axis=0)
            self.priors[c] = X_c.shape[0] / float(n_samples)
    
    def predict(self, X): # Para cada amostra em X, calcule a predição
        return np.array([self._predict(x) for x in X])
    
    def _predict(self, x): # Calculando a posteriori para cada classe
        posteriors = []

        for c in self.classes:
            
            prior = np.log(self.priors[c]) # Soma em logs para evitar underflow
            
            classe_condicionada = np.sum( np.log( self._pdf(c, x )) )
            
            posterior = prior + classe_condicionada
            
            posteriors.append(posterior)

        # retorna a classe mais provável
        return self.classes[np.argmax(posteriors)]
    
    def _pdf(self, c, x): # Calcula a função de densidade de probabilidade gaussiana
        
        mean = self.means[c] # Media da classe c
        
        var = self.vars[c] # Desvio Padrão da classe c
        
        # Evita var = 0
        var = np.maximum(var, 1e-8)

        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        
        denominator = np.sqrt(2 * np.pi * var)
        
        return numerator / denominator


# Item B

# 3) Classificador Bayesiano Multivariado

In [163]:
class NaiveBayesMultivariado:
    def _init_(self): 
        pass

    def fit(self, X, y):
        #Calcular médias, covariâncias e priori de cada classe
    
        self.X = np.array(X)
        self.y = np.array(y)
        self.classes = np.unique(y)
        
    # Dicionários que vão armazenar os parâmetros de cada classe:
        self.means = {}
        self.covs = {}
        self.priors = {}

# percorre cada classe para calcular seus parâmetros
        for c in self.classes:
            X_c = X[y == c]
            self.means[c] = X_c.mean(axis=0)
            self.covs[c] = np.cov(X_c, rowvar=False)
            self.priors[c] = X_c.shape[0] / X.shape[0]

    def _pdf_multivariada(self, x, mean, cov):
        
        d = len(mean)
        
        cov_det = np.linalg.det(cov) # Determinante da matriz de covariância
        
        cov_inv = np.linalg.inv(cov) # Inversa da matriz de covariância
        
        denominador = 1.0 / np.sqrt((2 * np.pi) ** d * cov_det) 
        
        x_menos_mean = np.transpose(x - mean) # Para fins de redução de cálculo na linha seguinte
        
        expoente = -0.5 * np.dot(np.dot(x_menos_mean, cov_inv), x_menos_mean.T)
        
        return float(denominador * np.exp(expoente.item()))


    def predict(self, X):
        
        X = np.array(X)
        
        y_pred = []
        
        for x in X:
            posteriors = []
            
            for c in self.classes:
                
                prior = np.log(self.priors[c]) # Todas amostras a prior das classes
                
                aproximacao = np.log(self._pdf_multivariada(x, self.means[c], self.covs[c]) + 1e-12) # Ao invés de multiplicar pequenas probabilidades, somamos os logaritmos para evitar underflow
                
                posterior = prior + aproximacao
                
                posteriors.append(posterior)
                
            y_pred.append(self.classes[np.argmax(posteriors)])
            
        return np.array(y_pred)


## Perceptron Simples

In [164]:
class PerceptronSimples:
    def __init__(self,learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None
        
    def fit(self, X, y):
        n_samples, n_features = X.shape #dimensoes
        self.weights = np.zeros(n_features)
        self.bias = 0 #inicializar os pesos e bias
        for _ in range(self.epochs):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights)+self.bias
                y_predicted = self._step_function(linear_output)
                if y[idx] != y_predicted:
                    update = self.learning_rate * (y[idx] - y_predicted)
                    self.weights += update *x_i
                    self.bias +=update
                    
    def _step_function(self,x):
        return np.where(x>=0, 1, 0)
    
    def predict(self, X):
        linear_output = np.dot(X, self.weights) +self.bias
        y_predicted = self._step_function(linear_output)
        return y_predicted

## MultiLayerPerceptron

In [165]:
class MLP:
    def __init__(self, n_inputs, n_hidden, learning_rate=0.01, epochs=2000):
        self.lr = learning_rate
        self.epochs = epochs
        
        # Inicialização dos pesos
        
        # Camada Oculta
        self.W1 = np.random.randn(n_inputs, n_hidden) * 0.01
        self.b1 = np.zeros((1, n_hidden)) # BIAS 
        
        # Camada de Saída
        self.W2 = np.random.randn(n_hidden, 1) * 0.01
        self.b2 = np.zeros((1, 1)) # BIAS

    def sigmoid(self, x): # Função de ativação sigmoide
        return 1 / (1 + np.exp(-x))

    def sigmoid_deriv(self, s): # Derivada da função sigmoide
        return s * (1 - s)

    def fit(self, X, y):
        m = X.shape[0]  # número de amostras
        
        for _ in range(self.epochs):
            # CAMADA OCULTA
            
            # FORWARD
            
            Z1 = np.dot(X, self.W1) + self.b1
            A1 = self.sigmoid(Z1)
            
            # CAMADA DE SAÍDA
            
            Z2 = np.dot(A1, self.W2) + self.b2
            A2 = self.sigmoid(Z2)

            # BACKPROPAGATION
            dZ2 = A2 - y.reshape(-1, 1)
            dW2 = np.dot(A1.T, dZ2) / m
            db2 = np.mean(dZ2, axis=0, keepdims=True)

            dA1 = np.dot(dZ2, self.W2.T)
            dZ1 = dA1 * self.sigmoid_deriv(A1) 
            dW1 = np.dot(X.T, dZ1) / m
            db1 = np.mean(dZ1, axis=0, keepdims=True)

            # AJUSTES DE PESO
            self.W1 -= self.lr * dW1
            self.b1 -= self.lr * db1
            self.W2 -= self.lr * dW2
            self.b2 -= self.lr * db2

    def predict(self, X):
        Z1 = np.dot(X, self.W1) + self.b1
        A1 = self.sigmoid(Z1)
        
        Z2 = np.dot(A1, self.W2) + self.b2
        A2 = self.sigmoid(Z2)
        
        return (A2 >= 0.5).astype(int).flatten()


# 4) Validaçao Cruzada “10-folds” (com random state=42) para avaliar e comparar os classificadores

In [ ]:
import time

class K_FOLD:
    def __init__(self, k, models: dict, seed=42):
        self.k = k
        self.models = models
        self.seed = seed
        np.random.seed(seed) # Seed para reprodutibilidade

    def split(self, X, Y):
        n_samples = X.shape[0]
        fold_size = n_samples // self.k
        indices = np.arange(n_samples)
        np.random.shuffle(indices) # Embaralha os índices dos dados aleatoriamente

        for i in range(self.k):
            
            start = i * fold_size
            
            end = start + fold_size if i != self.k - 1 else n_samples # Garante que o último fold pegue todos os dados restantes
            
            test_idx = indices[start:end]
            
            train_idx = np.concatenate((indices[:start], indices[end:]))
            
            yield X[train_idx], X[test_idx], Y[train_idx], Y[test_idx] # Mantem a função meio que pausada esperando algum retorno

    def accuracy(self, Y_true, Y_pred):
        return np.sum(Y_true == Y_pred) / len(Y_true)

    def precision(self, Y_true, Y_pred):
        precisions = []
        classes = np.unique(Y_true)

        for c in classes:
            tp = np.sum((Y_true == c) & (Y_pred == c))
            pred_pos = np.sum(Y_pred == c)
            prec = tp / pred_pos if pred_pos > 0 else 0
            precisions.append(prec)

        return precisions

    def recall(self, Y_true, Y_pred):
        recalls = []
        classes = np.unique(Y_true)
        for c in classes:
            tp = np.sum((Y_true == c) & (Y_pred == c))
            total_real = np.sum(Y_true == c)
            recalls.append(tp / total_real if total_real > 0 else 0)
        return recalls

    def f1_score(self, Y_true, Y_pred):
        precisao = np.mean(self.precision(Y_true, Y_pred))

        recall = np.mean(self.recall(Y_true, Y_pred))
        
        if (precisao + recall) == 0:
            return 0
        return 2 * (precisao * recall) / (precisao + recall)

    def evaluate(self, model_name: str, X, Y):
        model_data = self.models[model_name]

        acc_scores = []
        prec_scores = []
        f1_scores = []
        train_times = []
        test_times = []
        
        for X_train, X_test, Y_train, Y_test in self.split(X, Y):
            
            # Cria um modelo novo (zerado) usando a factory
            model = model_data["factory"]()

            # Treino
            t0 = time.time()
            model.fit(X_train, Y_train)
            train_time = time.time() - t0

            # Teste
            t1 = time.time()
            preds = model.predict(X_test)
            test_time = time.time() - t1
            
            # Métricas
            acc = self.accuracy(Y_test, preds)
            # print(acc)
            prec = self.precision(Y_test, preds)
            # print(preds)
            f1 = self.f1_score(Y_test, preds)
            # print(f1)


            acc_scores.append(acc)
            prec_scores.append(prec)
            f1_scores.append(f1)
            train_times.append(train_time)
            test_times.append(test_time)
            

    
        #  Resultados finais - Medias
        model_data["acuracia_media"] = np.mean(acc_scores)
        model_data["precisao_media"] = np.mean(prec_scores)
        model_data["f1_score_medio"] = np.mean(f1_scores)

        #  Resultados finais - DP
        model_data["dp_acuracia"] = np.std(acc_scores)
        model_data["dp_precisao"] = np.std(prec_scores)
        model_data["dp_f1"] = np.std(f1_scores)
        
        #  Resultados finais - Tempos
        model_data["tempo_treino"] = np.mean(train_times)
        model_data["tempo_teste"] = np.mean(test_times)

        print(f"\nModelo: {model_name}")
        print(f"Acurácia média: {model_data['acuracia_media']:.4f}")
        print(f"Variância acurácia: {model_data['dp_acuracia']:.4f}")
        print(f"Precisão média: {model_data['precisao_media']:.4f}")
        print(f"Variância precisão: {model_data['dp_precisao']:.4f}")
        print(f"F1-score médio: {model_data['f1_score_medio']:.4f}")
        print(f"Variância F1-score: {model_data['dp_f1']:.4f}")
        print(f"Tempo médio treino: {model_data['tempo_treino']:.4f}s")
        print(f"Tempo médio teste: {model_data['tempo_teste']:.4f}s")

        return model_data


In [167]:
UNI_K = 5
UNI_EPOCHS = 1000
UNI_LEARNING_RATE = 0.001

models = {
    "KnnEuclidean": {
        "factory": lambda: KNN(k=UNI_K, method="euclidean"), 
        "acuracia_media" : None,
        "dp_acuracia" : None,
        "precisao_media" : None,
        "dp_precisao" : None, 
        "f1_score_medio" : None,
        "dp_f1" : None, 
        "tempo_teste" : None, 
        "tempo_treino" : None
        },
    "KnnManhattan": {
        "factory": lambda: KNN(k=UNI_K, method="manhattan"), 
        "acuracia_media" : None,
        "dp_acuracia" : None,
        "precisao_media" : None,
        "dp_precisao" : None, 
        "f1_score_medio" : None,
        "dp_f1" : None, 
        "tempo_teste" : None, 
        "tempo_treino" : None
        },
    "NaiveBayesUnivariado": {
        "factory": lambda: NaiveBayesUnivariado(), 
        "acuracia_media" : None,
        "dp_acuracia" : None,
        "precisao_media" : None,
        "dp_precisao" : None, 
        "f1_score_medio" : None,
        "dp_f1" : None, 
        "tempo_teste" : None, 
        "tempo_treino" : None
        },
    "NaiveBayesMultivariado": {
        "factory": lambda: NaiveBayesMultivariado(), 
        "acuracia_media" : None,
        "dp_acuracia" : None,
        "precisao_media" : None,
        "dp_precisao" : None, 
        "f1_score_medio" : None,
        "dp_f1" : None, 
        "tempo_teste" : None, 
        "tempo_treino" : None
        },
    "PerceptronSimples": {
        "factory": lambda: PerceptronSimples(learning_rate=UNI_LEARNING_RATE, epochs=UNI_EPOCHS), 
        "acuracia_media" : None,
        "dp_acuracia" : None,
        "precisao_media" : None,
        "dp_precisao" : None, 
        "f1_score_medio" : None,
        "dp_f1" : None, 
        "tempo_teste" : None, 
        "tempo_treino" : None
    },
    "MultiLayerPerceptron": {
        "factory": lambda: MLP(n_inputs=X.shape[1], n_hidden=4, learning_rate=UNI_LEARNING_RATE, epochs=UNI_EPOCHS), 
        "acuracia_media" : None,
        "dp_acuracia" : None,
        "precisao_media" : None,
        "dp_precisao" : None, 
        "f1_score_medio" : None,
        "dp_f1" : None, 
        "tempo_teste" : None, 
        "tempo_treino" : None
    }
}

In [168]:
#Executando K-Fold Cross Validation
k_FOLD = K_FOLD(k=10, models=models)

for model_name in models.keys():
    k_FOLD.evaluate(model_name,X ,Y)

[[np.float64(0.4), np.float64(0.7241379310344828)], [np.float64(0.5), np.float64(0.7380952380952381)], [np.float64(0.5714285714285714), np.float64(0.788235294117647)], [np.float64(0.5555555555555556), np.float64(0.7469879518072289)], [np.float64(0.0), np.float64(0.75)], [np.float64(0.45454545454545453), np.float64(0.7777777777777778)], [np.float64(0.6), np.float64(0.7931034482758621)], [np.float64(0.4444444444444444), np.float64(0.7831325301204819)], [np.float64(0.25), np.float64(0.7386363636363636)], [np.float64(0.35714285714285715), np.float64(0.8470588235294118)]]

Modelo: KnnEuclidean
Acurácia média: 0.7408
Variância acurácia: 0.0267
Precisão média: 0.5910
Variância precisão: 0.2162
F1-score médio: 0.5642
Variância F1-score: 0.0605
Tempo médio treino: 0.0000s
Tempo médio teste: 0.2446s
[[np.float64(0.3333333333333333), np.float64(0.686046511627907)], [np.float64(0.2), np.float64(0.7701149425287356)], [np.float64(0.46153846153846156), np.float64(0.759493670886076)], [np.float64(0.5)

C:\Users\Lorde\AppData\Local\Temp\ipykernel_36616\3256339169.py:31: RuntimeWarning: divide by zero encountered in log
  classe_condicionada = np.sum( np.log( self._pdf(c, x )) )


[[np.float64(0.6470588235294118), np.float64(0.76)], [np.float64(0.8), np.float64(0.7816091954022989)], [np.float64(0.42857142857142855), np.float64(0.8)], [np.float64(0.6153846153846154), np.float64(0.7341772151898734)], [np.float64(0.6), np.float64(0.8658536585365854)], [np.float64(0.4117647058823529), np.float64(0.8266666666666667)], [np.float64(0.7142857142857143), np.float64(0.7764705882352941)], [np.float64(0.2717391304347826), 0], [np.float64(0.5333333333333333), np.float64(0.8051948051948052)], [np.float64(0.20202020202020202), 0]]

Modelo: PerceptronSimples
Acurácia média: 0.6604
Variância acurácia: 0.2144
Precisão média: 0.5787
Variância precisão: 0.2659
F1-score médio: 0.5540
Variância F1-score: 0.1843
Tempo médio treino: 2.8693s
Tempo médio teste: 0.0000s


C:\Users\Lorde\AppData\Local\Temp\ipykernel_36616\2982984344.py:17: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


[[0, np.float64(0.7608695652173914)], [0, np.float64(0.6956521739130435)], [0, np.float64(0.782608695652174)], [0, np.float64(0.6847826086956522)], [0, np.float64(0.75)], [0, np.float64(0.7282608695652174)], [0, np.float64(0.8043478260869565)], [0, np.float64(0.7282608695652174)], [0, np.float64(0.7282608695652174)], [0, np.float64(0.8484848484848485)]]

Modelo: MultiLayerPerceptron
Acurácia média: 0.7512
Variância acurácia: 0.0474
Precisão média: 0.3756
Variância precisão: 0.3771
F1-score médio: 0.4285
Variância F1-score: 0.0152
Tempo médio treino: 0.1121s
Tempo médio teste: 0.0001s
